In [116]:
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from Bio import SeqIO
import pandas as pd
import numpy as np

In [117]:
#读入文件（这个文件我暂时还没用上）
sequencesrna = []
for record in SeqIO.parse('sgRNA_reference.fa','fasta'):
    sequencesrna.append(str(record.seq))
pd.Series(sequencesrna)

0        TAGGAAGGCTTTGCATATT
1        TACTCCCTACAATCCATCT
2        ACAATCCATCTGGTCCCAA
3        ATGATACTGTCATCTTCTT
4        ACAGACGCCTCAGTCTCTG
                ...         
16320    CGTTTTTGCCCCCGATAAG
16321    CCGTAGCGGCGTAACGCGG
16322    GAGTCTTGATCTGCTCCCA
16323    GCGAGGGTGTCCTCCCATG
16324    AGTCCGGTTCTGCGGATGT
Length: 16325, dtype: object

In [118]:
#读入t 0时刻文件
sequences0 = []
for title, seq, qual in FastqGeneralIterator('t0.fastq'):
    #数据清理
    sequences0.append(seq[21:40])

In [119]:
#计算t 0时刻频数 
counts0=pd.Series(sequences0).value_counts()
counts0_sum=pd.Series(sequences0).value_counts().sum()

In [120]:
#读入t 10时刻文件
sequences10 = []
for title, seq, qual in FastqGeneralIterator('t10.fastq'):
    sequences10.append(seq[21:40])

In [121]:
#计算t 10时刻频数
counts10=pd.Series(sequences10).value_counts()
counts10_sum=pd.Series(sequences10).value_counts().sum()

In [122]:
#计算t 0时刻频率 
countsd0 = pd.DataFrame(counts0,columns=['counts0'])
countsd0['freq0']=counts0/counts0_sum

In [123]:
#计算t 10时刻频率 
countsd10=pd.DataFrame(counts10,columns=['counts10'])
countsd10['freq10']=counts10/counts10_sum

In [124]:
#合并两个表格
df_merge = countsd0.join(countsd10)

In [125]:
#计算变化率
df_merge['log2f']=np.log2(df_merge['freq0']/df_merge['freq10'])

In [126]:
df_merge.to_csv('result1.csv')